# bokeh prototype

This prototype uses `pandas-bokeh`. Another option is `hvplot`. Please see their respective documents. For sample recipes of both the packages, see also https://github.com/verdimrc/pyutil/blob/master/notebooks/.

In [ ]:
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

import bokeh as bk
import pandas as pd
import pandas_bokeh
from bokeh.layouts import gridplot, row, column
from bokeh.models import Div
from nbutils import Report, ReportIO

# Load an existing report. If not exist, create with 00_generate_report.ipynb.
report: Report = ReportIO('reports').load()

# Size of individual chart
figsize = (1280, 240)

In [ ]:
# Rewards chart.
def plot_rewards(rewards_list, **kwargs):
    df = pd.DataFrame({
        'rewards': rewards_list,
        'mean': sum(rewards_list) / len(rewards_list),
    })

    plot = df.plot_bokeh(
        kind="line",
        title="Rewards",
        xlabel="Episode",
        ylabel="Mean reward per episode",
        colormap=["blue", "red"],
        show_figure=False,
        **kwargs,
    )

    # We don't want tooltip on the mean line. Is there a better way than this hack?
    # NOTE: hvplot allows specifying df columns to enable hover using `hover_col`.
    #       See https://hvplot.holoviz.org/user_guide/Customization.html
    #       Seems like a good reason to move to hvplot (though I prefer pandas_bokeh's
    #       default layout better).
    plot.tools.remove(plot.tools[-1])

    return plot


bk.io.save(
    plot_rewards(report.rewards_list, figsize=figsize),
    filename='rewards.html',
    resources=bk.resources.INLINE,
    title='Rewards'
);

In [ ]:
# Evaluation charts.
def plot_analysis(df, **kwargs):
    kwargs = dict(**kwargs, show_figure=False)
    plots = [
        df[['cost', 'price']].plot_bokeh(kind='line', title='Cost vs Price', **kwargs), # TODO: off tooltip on all-but-one line.
        df[['action']].plot_bokeh(kind='scatter', title='Actions Taken', **kwargs),
        df[['reward']].plot_bokeh(kind='line', title='Rewards', **kwargs),
        df[['total_reward']].plot_bokeh(kind='line', title='Total Reward', **kwargs),
        df[['energy']].plot_bokeh(kind='line', title='Energy (Inventory Level)', **kwargs),
    ]
    return plots

grid_analysis = gridplot(
    plot_analysis(report.df_history, figsize=figsize),
    ncols=1,
    merge_tools=False,
)

bk.io.save(
    grid_analysis,
    filename='analysis.html',
    resources=bk.resources.INLINE,
    title='Analysis'
);

In [ ]:
# Save another all-in-one .html

# NOTE: with pandas-bokeh, save() requires new grid AND p_*, or else exception:
#
#     RuntimeError: Models must be owned by only a single document, SaveTool(id='1243', ...) is already in a doc
#
grid_all = column(
    Div(text="<h1>plot_rewards()</h1>"),
    plot_rewards(report.rewards_list, figsize=figsize),
    Div(),
    Div(text="<h1>plot_analysis()</h1>", sizing_mode='scale_width', width=figsize[0]),
    gridplot(plot_analysis(report.df_history, figsize=figsize), ncols=1, merge_tools=False),
)

bk.io.save(grid_all, filename='all.html', resources=bk.resources.INLINE, title='All-in-One');